**Lab - RNNs for Text Generation**

This lab was modified from an example originally written by "The TensorFlow Authors" in 2019 and distributed under Apache License 2.0. This lab creates a model that can generate text using a character-based RNN. A character-based RNN learns sequences of characters from a corpus of text. Once the model is trained, one can present an input character sequence and the model will generate the character that it predicts would be most likely to appear next. By repeatedly calling the model for new predictions with the previously built sequence, one can create a string of text that "looks like" sentences from the original training text. Note that depending upon the amount of training material and the details of the training procedure, this generated text may look more or less like gibberish.

Also note that, depending on the specific types of layers you are training, you might want to try enabling GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*. Do this before you start to run any code, because it generally restarts the runtime and your local variables will be lost.

## Setup

### Import TensorFlow and other libraries

In [1]:
pip install tensorflow

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow
Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os # To access local files; for saving checkpoints
import time

from urllib import request # We will need this to read from an URL

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
# TensorFlow 2.0 offers "Eager Execution," a more practical model for 
# running tf code. Are we using it here?
tf.executing_eagerly()

True

### Download the Text Data

Load a plain text book from Project Gutenberg:

In [3]:
# Some collected plays by Anton Chekhov
url = "https://www.gutenberg.org/files/7986/7986-0.txt"
response = request.urlopen(url)
text = response.read().decode('utf8')
type(text), len(text)

(str, 411576)

In [4]:
# Additional URLs of plain text Chekhov plays for later use
uncle_vanya = "https://www.gutenberg.org/cache/epub/1756/pg1756.txt"
the_seagull = "https://www.gutenberg.org/files/1754/1754-0.txt"

### Read the data

First, look in the text to see what we have. Note that this is a character-based model, so we are interested to know what different characters are used throughout the whole text. Note that this should make the process largely language independent: Any language where words are formed through sequences of characters should work in training this kind of model.

In [5]:
# The unique text characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

94 unique characters


## Process the text

### Vectorize the text

Before training, we will convert the strings to a numerical representation. The approach in this notebook uses a so-called "ragged tensor." Here's an excerpt from the TensorFlow documentation:

"Ragged tensors are the TensorFlow equivalent of nested variable-length lists. They make it easy to store and process data with non-uniform shapes, including: Variable-length features, such as the set of actors in a movie; Batches of variable-length sequential inputs, such as sentences or video clips; Hierarchical inputs, such as text documents that are subdivided into sections, paragraphs, sentences, and words."

In the cells below, the notebook uses `preprocessing.StringLookup` from TensorFlow. This function can convert each character into a numeric ID. The input to this is a set of "byte code" ID numbers.

Try the character encoding with a small example first:

In [6]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
print(chars) # In TF 2.0, the print() function can also show tensors

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>


We have now created a ragged tensor. Now or later, you might want to read some of the TensorFlow documentation describing what a tensor actually is and what it means to be ragged. There's a nice tutorial with code here: https://www.tensorflow.org/guide/tensor

Note that each element in the structure above begins with b. That means that the function has returned a "byte-coded" version of the string. This is an alternative representation to plain text. Also, the idea of "ragged" is demonstrated by this example. The first nested list has seven items, whereas the second list has three. Ragged means that different numbers of elements are permissible within each element of a list. The term ragged comes from publishing, where a common way of justifying text in a book is called "ragged right" (meaning that different lines are different lengths).

Next, create a preprocessing.StringLookup layer. Note this initial diagnostic test to show what preprocessing.StringLookup actually is. It is an object reflecting a relatively new feature of Python called an "abstract base class."

In [10]:
# Note that we are passing in the vocab from parsing the whole book in an 
# earlier cell. Examine the first argument closely:
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)

# Shows the resulting class type and that we now have an instance
type(ids_from_chars), isinstance(ids_from_chars, preprocessing.StringLookup)

(keras.layers.preprocessing.string_lookup.StringLookup, True)

Our instances of the preprocessing.StringLookup ABC can convert from byte coded character tokens to numeric character IDs:

In [11]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[59, 60, 61, 62, 63, 64, 65], [82, 83, 84]]>

In the previous exercise you retrieved the character representations through direct indexing into the vocabulary list, but you can also use `preprocessing.StringLookup(..., invert=True)`. Note: Make sure to use the get_vocabulary() method of the preprocessing.StringLookup layer so that the [UNK] tokens (if any) are appropriately labeled.

In [14]:
# This creates an instance of the "inverter"
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

chars = chars_from_ids(ids) # Now use the inverter to process our tiny example
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Note that we have gotten back the byte codes of the characters from the vectors of IDs, and we are seeing them as a `tf.RaggedTensor` of characters. We can now use another utility, tf.strings.reduce_join, to join the characters back into strings. This is demonstrated here because it will be helpful later when we want to generate new text.

In [13]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [15]:
# Let's turn that utility into a function that we can call.
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task that our RNN model will perform. The input to the model will be sequences of characters from the book we read in at the top of the notebook.


### Create training examples and targets

Next, we will divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text. For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello". This ensures that we have tons of training examples and that each training example captures context both around the input string and the output string.

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [16]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
type(all_ids)

tensorflow.python.framework.ops.EagerTensor

In [17]:
# This creates a dataset whose elements are slices from the original tensor
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
type(ids_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [18]:
#
#  Run this code and look at the output that it 
# generates. Check out what each line of the code to say what it is doing.

temp = ids_dataset.unique()
for element in temp:
  print(element.numpy())

#
#

94
45
76
73
68
63
61
78
3
36
79
72
60
65
91
77
70
59
83
32
66
69
80
11
48
62
67
30
2
1
49
31
64
81
71
13
54
74
12
41
25
44
37
47
52
38
33
17
15
20
56
34
5
22
24
23
21
57
50
16
35
10
39
43
40
51
82
18
42
92
93
8
9
75
19
26
84
28
46
27
14
55
4
88
85
53
58
87
90
86
89
7
29
6


In [19]:
# The take() bound method is helpful for examining data in a tensor. It is 
# modeled after the take() methods from numpy.
for ids in ids_dataset.take(18):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

﻿
P
r
o
j
e
c
t
 
G
u
t
e
n
b
e
r
g


In [20]:
#
# Add the skip() bound method to the expression in the for loop
# in the cell just above. To demonstrate how it works, skip 18 elements and then
# take the next 10 elements. The invocation of skip() in the expression should
# precede the invocation of take().
#
for ids in ids_dataset.skip(18).take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

’
s
 
P
l
a
y
s
 
b


In [21]:
# While an RNN can theoretically handle a continuous stream of data
# here we are considering the data in small groupings whose length
# is controlled by seq_length. Leave it at its current value for now, but 
# in the future you might consider making it either shorter or longer in the
# training run.
seq_length = 80 # About one line of standard text
examples_per_epoch = len(text)//(seq_length+1)
print(examples_per_epoch)

5081


In [22]:
#
#  Explain why seq_length+1 is used in the next line of code
#
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(3):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\xef\xbb\xbf' b'P' b'r' b'o' b'j' b'e' b'c' b't' b' ' b'G' b'u' b't'
 b'e' b'n' b'b' b'e' b'r' b'g' b'\xe2\x80\x99' b's' b' ' b'P' b'l' b'a'
 b'y' b's' b' ' b'b' b'y' b' ' b'C' b'h' b'e' b'k' b'h' b'o' b'v' b','
 b' ' b'S' b'e' b'c' b'o' b'n' b'd' b' ' b'S' b'e' b'r' b'i' b'e' b's'
 b',' b' ' b'b' b'y' b' ' b'A' b'n' b't' b'o' b'n' b' ' b'C' b'h' b'e'
 b'k' b'h' b'o' b'v' b'\r' b'\n' b'\r' b'\n' b'T' b'h' b'i' b's' b' ' b'e'
 b'B'], shape=(81,), dtype=string)
tf.Tensor(
[b'o' b'o' b'k' b' ' b'i' b's' b' ' b'f' b'o' b'r' b' ' b't' b'h' b'e'
 b' ' b'u' b's' b'e' b' ' b'o' b'f' b' ' b'a' b'n' b'y' b'o' b'n' b'e'
 b' ' b'a' b'n' b'y' b'w' b'h' b'e' b'r' b'e' b' ' b'a' b't' b' ' b'n'
 b'o' b' ' b'c' b'o' b's' b't' b' ' b'a' b'n' b'd' b' ' b'w' b'i' b't'
 b'h' b'\r' b'\n' b'a' b'l' b'm' b'o' b's' b't' b' ' b'n' b'o' b' ' b'r'
 b'e' b's' b't' b'r' b'i' b'c' b't' b'i' b'o' b'n' b's'], shape=(81,), dtype=string)
tf.Tensor(
[b' ' b'w' b'h' b'a' b't' b's' b'o' b'e' b'v' b'e' b'r' b'

It's easier to see what this is doing if you join the tokens back into strings:

In [23]:
for seq in sequences.take(2):
  print(text_from_ids(seq).numpy())

b'\xef\xbb\xbfProject Gutenberg\xe2\x80\x99s Plays by Chekhov, Second Series, by Anton Chekhov\r\n\r\nThis eB'
b'ook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions'


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. Here's a function that takes a sequence as input, duplicates it, and shifts it to align the input and label for each timestep:

In [24]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [25]:
# Let's test the function - We can do the test using
# a regular character string converted to a list.
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [26]:
# This is a curious construction - i.e., passing a function into the
# sequences.map() bound method:
dataset = sequences.map(split_input_target)
#
#

In [27]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\xef\xbb\xbfProject Gutenberg\xe2\x80\x99s Plays by Chekhov, Second Series, by Anton Chekhov\r\n\r\nThis e'
Target: b'Project Gutenberg\xe2\x80\x99s Plays by Chekhov, Second Series, by Anton Chekhov\r\n\r\nThis eB'


### Create training batches

We have used `tf.data` to split the text into manageable sequences. Before using these data to train the model, we need to shuffle the data and pack it into batches. AKA mini-batching, is a method of processing a group of input-output pairs together. This facilitates parallelization and can prevent overfitting.

In [28]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset:
# TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements.
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 80), dtype=tf.int64, name=None), TensorSpec(shape=(64, 80), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.SimpleRNN`: A type of RNN with size `units=rnn_units` (You could also use a GRU or LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [29]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256 # 256 is kind of a generic choice that should work well
# for various-sized character vocabularies. There is a danger of overfitting 
# when the size of the embedding layer exceeds the size of the vocabulary.

# Number of RNN units
rnn_units = 1024 # This is tunable. Remember that each RNN unit has a little
# bit of memory for what came before. Here 1024 provides four nodes for every
# node in the embedding layer. Later, you might want to experiment with half
# as many or twice as many.

vocab_size, embedding_dim, rnn_units

(94, 256, 1024)

In [31]:
# This builds a custom class for instantiating the Keras model
#
#
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)

    # What's this layer?
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    
    # What's this layer?
    self.rnn = tf.keras.layers.SimpleRNN(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    
    # What's this layer?
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.rnn.get_initial_state(x)
    x, states = self.rnn(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

We could have used a `keras.Sequential` model here, as this architecture is quite simple. However, to  generate text later we will need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

In [32]:
# Now instantiate the class defined above.

model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the RNN one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character. A logit can be turned into an odds ratio with exponentiation. Statisticians like to work with logits because they behave linearly. 

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [33]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 80, 95) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `80` but the model can be run on inputs of any length:

In [34]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  24320     
                                                                 
 simple_rnn (SimpleRNN)      multiple                  1311744   
                                                                 
 dense (Dense)               multiple                  97375     
                                                                 
Total params: 1,433,439
Trainable params: 1,433,439
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we should sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary. The TF documentation says that it is important to _sample_ from this distribution as taking the _argmax_ of the distribution because that can easily get the model stuck in a loop. 

Taking the argmax means that the text generator will make the same choice every time when confronted with a given input. As a result, we could present "to be" as the input and get "to be or not to be or not to be or not to be. . ." as the output. By sampling from the output instead, we can get a variety of somewhat random responses each time we present the input, so "to be" could generate the response, "to be or not to be, that is the snorgle."

Try it for the first example in the batch:

In [35]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index. You should run the cell above and the cell below several times:

In [36]:
sampled_indices

array([59, 18, 59, 48, 73, 49, 91, 74, 22, 60, 70, 26, 71,  0, 46, 12, 12,
       68, 52, 60, 29, 52, 37, 32, 52, 78, 14, 58, 39,  8, 92, 15, 51, 80,
       85, 50, 27, 65, 91, 16, 44, 47, 86,  3,  2, 37,  9, 16, 29, 78, 16,
       29, 21, 23, 56, 62,  4, 30, 58, 26, 17, 54,  7, 34, 55, 38, 54, 59,
       93, 64, 48, 81, 59, 46, 94, 81, 86, 14, 31, 55])

Decode these to see the text predicted by this untrained model:

In [37]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'iling\r\nin my chest! Uncle Tihon! [Weeps] Uncle Tihon!\r\n\r\nSAWA. [Groans] I\xe2\x80\x99ve got'

Next Char Predictions:
 b'a3aSoT\xe2\x80\x99p7bl;m[UNK]Q--jWb@WHCWt/_J(\xe2\x80\x9c0Vv\xc2\xa3U=g\xe2\x80\x991OR\xc3\xa0 \rH)1@t1@68[d!A_;2Y%EZIYa\xe2\x80\x9dfSwaQ\xef\xbb\xbfw\xc3\xa0/BZ'


Naturally, the predicted text is garbage because the untrained model essentially makes random predictions.

## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [38]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [39]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 80, 95)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.564593


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [40]:
tf.exp(mean_loss).numpy()

96.02348

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [41]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [42]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 20 epochs to train the model. In Colab, set the runtime to GPU for faster training. Note that with all of the starter defaults in this notebook, each epoch takes about 10 seconds, so only about 3 minutes to train this model. 

In [43]:
EPOCHS = 20

In [44]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
79/79 [==============================] - 12s 116ms/step - loss: 3.2642
Epoch 2/20
79/79 [==============================] - 10s 116ms/step - loss: 2.4262
Epoch 3/20
79/79 [==============================] - 11s 123ms/step - loss: 2.1968
Epoch 4/20
79/79 [==============================] - 10s 116ms/step - loss: 2.0366
Epoch 5/20
79/79 [==============================] - 10s 118ms/step - loss: 1.9064
Epoch 6/20
79/79 [==============================] - 10s 115ms/step - loss: 1.8017
Epoch 7/20
79/79 [==============================] - 10s 117ms/step - loss: 1.7141
Epoch 8/20
79/79 [==============================] - 11s 126ms/step - loss: 1.6368
Epoch 9/20
79/79 [==============================] - 17s 203ms/step - loss: 1.5733
Epoch 10/20
79/79 [==============================] - 11s 116ms/step - loss: 1.5159
Epoch 11/20
79/79 [==============================] - 10s 117ms/step - loss: 1.4718
Epoch 12/20
79/79 [==============================] - 10s 117ms/step - loss: 1.4295
Epoch 13/20
7

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as it runs. Each time we call the model we pass in a slice of text and an internal state. 

The model returns a prediction for the next character as well as its new state. Pass the prediction and state back in to continue generating text. The class defined below accomplishes one step in this chain of model runs. When the generate_one_step() bound method is called, it makes a single step prediction. Note the temperature argument on the class initializer.


In [45]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [46]:
# Now we can instantiate the class. Take note of the arguments we are 
# passing in. 
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Chekhov-like  vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [51]:
start = time.time()
states = None
next_char = tf.constant(['NATALYA STEPANOVNA.'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

NATALYA STEPANOVNA. Masha. And what is it? Bat what all sit it, how much or orchard woss grout I get an on, I must
come, too.... Just orrew.... What a drivalinuties and your grandfather; TATIANA ALEXEYEVNA. Oh, if a starc works of anybody else.... There’s to  runnafe.

LUKA. [Gets and many)! [Downes up and first kippe?

LOMOV. Like a provincies and teensbout to beoure je
and so
at the state exchamms, he dod. Ourse, which Creating strumping and said cole in. Seeter] Timply here upper. I can’t...

LUBOV. How months
tickens
everything hashes. Good-bye, and
then are you?

MERCHUTKINA. You’re jacket and real it’s put thus.
[Sits into your exituse-topares. A porsed to me?

SOLENI. [Firgars here!

VERSHININ. Thank you.... It’s allow me beins sook]
[Attaciater after share earth. There’s out with a fat here, I’ll cry qull housing it sound it’s hard so
your axe of the drawinable for the darne. Is if you roubles
for if you dear. [Looks a marroly.

CHUBUKOV. I only got an ehtremee.... Oh, on  

__

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text faster the easiest thing you can do is batch the text generation. In the example below the model generates three texts in about the same time it took to generate just one above.

In [52]:
start = time.time()
states = None
next_char = tf.constant(['NATALYA STEPANOVNA.', 'NATALYA STEPANOVNA.', 'NATALYA STEPANOVNA.'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'NATALYA STEPANOVNA. The stage up byer, don\xe2\x80\x99t have pethement women.\r\n\r\nCHEBUTIKIN. So that if I don\xe2\x80\x99t like a great be\r\nme the time wend son you; corre-tiom, whatereduing about eighto man hunured\r\nhim thir? That\xe2\x80\x99s earie coming four rusbance....\r\n\r\nSHIPUCHIN. [Clangurt, as he day and appross. Fill, I can\xe2\x80\x99t four, \xe2\x80\x9cKharkov Andreyevitch, let\xe2\x80\x99s sinducals. Enter grinoms at all mat\r\nlater on.... And when There is is that you shouldn\xe2\x80\x99t off from any more, and heard peepiciture now. [Pause.]\r\n\r\nANFISA. Thronifella most examp. [Covers her boseken.\r\n\r\nKULIGIN. Natashs.... Yes, riall.\r\n[IRINA SEDREYEVNA] Nive you got into a low! [To YASHA] Mather Feers.\r\n\r\nOLGA. It\xe2\x80\x99s a counter.\r\n\r\nTRSMA. Sold woman who\r\nted him; it\xe2\x80\x99s sign of the Project Gutenberg-tm.... [Chappears] Bar\r\ntime I\xe2\x80\x99m qulence.... [At the brother here, or\r\nkits himself\r\na child o

**Improving the Model**

There are several strategies that might improve the performance of the model. Try them in the following order:

* Reduce the temperature in the initialization of the OneStep class to reduce the randomness in generation of new characters 
* Increase the training epochs by 50% or more to get the model loss to be lower
* Increase the number of nodes in the existing SimpleRNN layer to give the model more "intelligience"
* Add an additional layer of RNN after the first layer to improve the model's "memory"

Depending upon the amount of time left in the lab, try one or more of these techniques. For the moment we don't have a way of documenting model quality other than your own read of the generated text to see whether it is creating real words and sensible sentences. But that's a fine criterion for now. Make sure to add comments documenting what you find out.

## Advanced: Customized Training

The following material is for strudents who want to dive more deeply into TensorFlow. If you have run out of options for modifying and testing the simple model above and there is time left in the lab, you may want to learn more about improving the quality of a character-based text generation model. Examine the customized training class provided below and spend some time thinking about how it works. 

All the training we have done to this point is very straightforward and is no different from outher TF/Keras models we've experimented with. Some would call this an open-loop model, because prediction mistakes are not fed back into the model to improve it. You might want to consult this article first: https://towardsdatascience.com/what-is-teacher-forcing-3da6217fed1c

The custom class below improves over our previous training method by closing the "open loop" that allowed prediction errors to accumulate. The most important part of a custom training loop is the train step function.

The loop uses `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [53]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [54]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [55]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [56]:
model.fit(dataset, epochs=1)

79/79 [==============================] - 11s 117ms/step - loss: 3.2774


Or if you need more control, you can write your own complete custom training loop:

In [57]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.6825
Epoch 1 Batch 50 Loss 2.3624

Epoch 1 Loss: 2.4310
Time taken for 1 epoch 9.95 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 2.2413
Epoch 2 Batch 50 Loss 2.1659

Epoch 2 Loss: 2.1882
Time taken for 1 epoch 9.27 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 2.0044
Epoch 3 Batch 50 Loss 2.0007

Epoch 3 Loss: 2.0248
Time taken for 1 epoch 10.24 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.9604
Epoch 4 Batch 50 Loss 1.9587

Epoch 4 Loss: 1.8972
Time taken for 1 epoch 9.16 sec
________________________________________________________________________________
Epoch 5 Batch 0 Loss 1.8307
Epoch 5 Batch 50 Loss 1.8021

Epoch 5 Loss: 1.7876
Time taken for 1 epoch 9.28 sec
________________________________________________________________________________
Epoch 6 Batch 0 Loss 1.7330
Epoch 6 Bat

In [ ]:
#
# This training code created a trained "model" object, just as the earlier, 
# simpler code did. So you can use all of the predictive machinery that
# appeared earlier in this notebook to test the results.
#